In [1]:
library(yaml)
library(COMBINEHarmonizer)
library(readxl)
library(dplyr)
library(data.table)


Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag




The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





Attaching package: ‘data.table’




The following objects are masked from ‘package:dplyr’:

    between, first, last




## 00. Read Config

In [2]:
root_dir <- "../"

cfg_filename <- file.path(root_dir, "config.yaml")
cfg <- yaml::read_yaml(cfg_filename)

In [3]:
input_dir <- file.path(cfg$out_dir, "stats")
out_dir <- file.path(cfg$out_dir, "stats")

# if (!file.exists(out_dir)) {
#     dir.create(out_dir, recursive = TRUE)
# }

data_dict_filename <- file.path(root_dir, COMBINEHarmonizer::DATA_DICTIONARY_EXCEL)

In [4]:
df_data_dict_main <- COMBINEHarmonizer::load_data_dict(
    data_dict_filename,
    COMBINEHarmonizer::SHEET_MAIN
)
df_data_dict_followup <- COMBINEHarmonizer::load_data_dict(
    data_dict_filename,
    COMBINEHarmonizer::SHEET_FOLLOW_UP
)

In [5]:
data_dict_main_map <- df_data_dict_main[[COMBINEHarmonizer::DATA_DICT_VAR_TYPE]]
names(data_dict_main_map) <- df_data_dict_main[[COMBINEHarmonizer::DATA_DICT_VAR_NAME]]

data_dict_followup_map <- df_data_dict_followup[[COMBINEHarmonizer::DATA_DICT_VAR_TYPE]]
names(data_dict_followup_map) <- df_data_dict_followup[[COMBINEHarmonizer::DATA_DICT_VAR_NAME]]

DATA_DICT_MAP <- list()
DATA_DICT_MAP[[COMBINEHarmonizer::SHEET_MAIN]] <- data_dict_main_map
DATA_DICT_MAP[[COMBINEHarmonizer::SHEET_FOLLOW_UP]] <- data_dict_followup_map


In [6]:
FILENAME_INFO_DATA_DICT_MAP <- list()
for (filename_info in COMBINEHarmonizer::FILENAME_INFOS) {
    each_name <- filename_info$name
    each_data_dict <- filename_info$data_dict
    FILENAME_INFO_DATA_DICT_MAP[[each_name]] <- DATA_DICT_MAP[[each_data_dict]]
}

PREFIX_DATA_DICT_MAP <- list()
for (each_filename in names(FILENAME_INFO_DATA_DICT_MAP)) {
    each_val <- FILENAME_INFO_DATA_DICT_MAP[[each_filename]]
    each_flatten_prefix <- COMBINEHarmonizer::flatten_filename_prefix(each_filename)

    message(c("[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: ", each_filename, " prefix: ", each_flatten_prefix))
    PREFIX_DATA_DICT_MAP[[each_flatten_prefix]] <- each_val
}

[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 00-02-screening.csv prefix: 00-02



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 00-12-neuro-exam.csv prefix: 00-12



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 01-02-screening.csv prefix: 01-02



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 01-03-maternal-demographics.csv prefix: 01-03



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 01-04-pregnancy-history.csv prefix: 01-04



...

[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 20-07-readmission.csv prefix: 20-07



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 20-08-lost.csv prefix: 20-08



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 20-09-secondary.csv prefix: 20-09



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 20-10-outcome.csv prefix: 20-10



[INFO] after COMBINEHarmonizer::flatten_filename_prefix: filename: 20-10_1-disability-level-death.csv prefix: 20-10_1



## Load flatten

In [7]:
filename <- file.path(input_dir, "zz-merged-flatten-numeric.csv")


In [8]:
df_numeric <- read.csv(filename)

### 03. Construct df_numeric

In [9]:
columns <- names(df_numeric)
valid_columns <- c()
for (column in columns) {
    column_tuple <- COMBINEHarmonizer::flatten_column_tuple(column)
    prefix <- column_tuple[[1]]
    var_name <- column_tuple[[2]]
    if (!prefix %in% names(PREFIX_DATA_DICT_MAP)) {
        message(c("[WARN] not in PREFIX_DATA_DICT_MAP: column: ", column, " prefix: ", prefix))
        next
    }

    data_dict_map <- PREFIX_DATA_DICT_MAP[[prefix]]
    if (!var_name %in% names(data_dict_map)) {
        message(c("[WARN] not in DATA_DICT_MAP: var_name: ", var_name, " prefix: ", prefix))
        next
    }

    valid_columns <- c(valid_columns, column)
}

df_numeric_valid <- df_numeric[, valid_columns, drop = FALSE]

[WARN] not in PREFIX_DATA_DICT_MAP: column: X_study prefix: 



[WARN] not in PREFIX_DATA_DICT_MAP: column: center prefix: 



[WARN] not in PREFIX_DATA_DICT_MAP: column: subjectID prefix: 



[WARN] not in PREFIX_DATA_DICT_MAP: column: uniqueID prefix: 



[WARN] not in DATA_DICT_MAP: var_name: MRI_ID prefix: 03-05



...

[WARN] not in DATA_DICT_MAP: var_name: uniqueFollowupID prefix: 20-05



[WARN] not in DATA_DICT_MAP: var_name: uniqueFollowupID prefix: 20-06



[WARN] not in DATA_DICT_MAP: var_name: uniqueFollowupID prefix: 20-08



[WARN] not in DATA_DICT_MAP: var_name: uniqueFollowupID prefix: 20-09



[WARN] not in DATA_DICT_MAP: var_name: uniqueFollowupID prefix: 20-10



In [10]:
valid_columns

[1] "X01.02.siteID"                                                      
   [2] "X01.02.birthNumber"                                                 
   [3] "X01.02.screenComment"                                               
   [4] "X01.02.coreTempLess32p5CGreaterEq2Hr_e"                             
   [5] "X01.02.coreTempLess33p5CGreater1Hr_e"                               
   [6] "X01.02.coreTempLess34CGreater1Hr_e"                                 
   [7] "X01.02.first6HrCoolByClinicalProtocol_e"                            
   [8] "X01.02.chromosomalAbnormality_e"                                    
   [9] "X01.02.majorCongenitalAnomaly_e"                                    
  [10] "X01.02.birthWeightLessEq1800g_e"                                    
  [11] "X01.02.infantUnlikelySurvive_e"                                     
  [12] "X01.02.first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e"
  [13] "X01.02.postnatalAgeLess6HrOrGreater24Hr_e"                          
  [14] "X01.02.enrolledConflictingTrial_e"                                  
  [15] "X01.02.first60MinAnyBloodGasPHLessEq7_i"                            
  [16] "X01.02.first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i"        
  [17] "X01.02.historyPerinatalEvent_i"                                     
  [18] "X01.02.at10MinApgarLessEq5OrVent_i"                                 
  [19] "X01.02.randomEligible"                                              
  [20] "X01.02.consentStatus"                                               
  [21] "X01.02.noConsentReason"                                             
  [22] "X01.02.noInStudyReason"                                             
  [23] "X01.02.random"                                                      
  [24] "X01.02.noRandomReason"                                              
  [25] "X01.02.noRandomReasonText"                                          
  [26] "X01.02.randomDate"                                                  
  [27] "X01.02.randomTime"                                                  
  [28] "X01.02.ageRand_hr"                                                  
  [29] "X01.02.randomNumber"                                                
  [30] "X01.02.randomTreatmentAssign"                                       
  [31] "X01.02.randomTreatmentReceive"                                      
  [32] "X01.02.treatmentBlanketType"                                        
  [33] "X01.02.treatmentAssignmentDuration_hr"                              
  [34] "X01.02.treatmentAssignmentTemperature"                              
  [35] "X01.02.usualCoolingTreatmentGroup"                                  
  [36] "X01.02.inOtherTrial"                                                
  [37] "X01.02.inOtherTrialText"                                            
  [38] "X01.02.birthDate"                                                   
  [39] "X01.03.motherAge_year"                                              
  [40] "X01.03.motherRace"                                                  
  [41] "X01.03.motherRaceOther1"                                            
  [42] "X01.03.motherRaceOther2"                                            
  [43] "X01.03.motherRaceOther3"                                            
  [44] "X01.03.motherRaceOther4"                                            
  [45] "X01.03.motherRaceOther5"                                            
  [46] "X01.03.motherRaceOther6"                                            
  [47] "X01.03.motherRace2"                                                 
  [48] "X01.03.motherEthnicity"                                             
  [49] "X01.03.motherMaritalStatus"                                         
  [50] "X01.03.motherEducation"                                             
  [51] "X01.03.motherEducation2"                                            
  [52] "X01.03.motherInsurance"                                             

In [11]:
length(names(df_numeric_valid))

[1] 6107

## 03. all

In [12]:
y_columns <- c(
    "X20.10.moderateSevereDisabilityOrDeath",
    "X20.10.disabilityLevelDeath4Category",
    "X20.10_1.disabilityLevelDeath",

    "X20.04.BayleyIIICognitiveComposite",
    "X20.04.BayleyIIILanguageComposite",
    "X20.04.BayleyIIIMotorComposite"
)

value_columns <- c(
    "corr",
    "r2",
    "pvalue",
    "valid",
    "valid_x",
    "valid_y",
    "valid_percent"
)

to_csv_column <- function(y_column, value_column) {
    paste(c(y_column, value_column), collapse = " ")
}

reorder_columns <- function(y_columns, value_columns) {
    ret <- c()
    for (y_column in y_columns) {
        for (value_column in value_columns) {
            ret <- c(ret, to_csv_column(y_column, value_column))
        }
    }

    return(ret)
}

reordered_columns <- reorder_columns(y_columns, value_columns)
reordered_columns

[1] "X20.10.moderateSevereDisabilityOrDeath corr"         
 [2] "X20.10.moderateSevereDisabilityOrDeath r2"           
 [3] "X20.10.moderateSevereDisabilityOrDeath pvalue"       
 [4] "X20.10.moderateSevereDisabilityOrDeath valid"        
 [5] "X20.10.moderateSevereDisabilityOrDeath valid_x"      
 [6] "X20.10.moderateSevereDisabilityOrDeath valid_y"      
 [7] "X20.10.moderateSevereDisabilityOrDeath valid_percent"
 [8] "X20.10.disabilityLevelDeath4Category corr"           
 [9] "X20.10.disabilityLevelDeath4Category r2"             
[10] "X20.10.disabilityLevelDeath4Category pvalue"         
[11] "X20.10.disabilityLevelDeath4Category valid"          
[12] "X20.10.disabilityLevelDeath4Category valid_x"        
[13] "X20.10.disabilityLevelDeath4Category valid_y"        
[14] "X20.10.disabilityLevelDeath4Category valid_percent"  
[15] "X20.10_1.disabilityLevelDeath corr"                  
[16] "X20.10_1.disabilityLevelDeath r2"                    
[17] "X20.10_1.disabilityLevelDeath pvalue"                
[18] "X20.10_1.disabilityLevelDeath valid"                 
[19] "X20.10_1.disabilityLevelDeath valid_x"               
[20] "X20.10_1.disabilityLevelDeath valid_y"               
[21] "X20.10_1.disabilityLevelDeath valid_percent"         
[22] "X20.04.BayleyIIICognitiveComposite corr"             
[23] "X20.04.BayleyIIICognitiveComposite r2"               
[24] "X20.04.BayleyIIICognitiveComposite pvalue"           
[25] "X20.04.BayleyIIICognitiveComposite valid"            
[26] "X20.04.BayleyIIICognitiveComposite valid_x"          
[27] "X20.04.BayleyIIICognitiveComposite valid_y"          
[28] "X20.04.BayleyIIICognitiveComposite valid_percent"    
[29] "X20.04.BayleyIIILanguageComposite corr"              
[30] "X20.04.BayleyIIILanguageComposite r2"                
[31] "X20.04.BayleyIIILanguageComposite pvalue"            
[32] "X20.04.BayleyIIILanguageComposite valid"             
[33] "X20.04.BayleyIIILanguageComposite valid_x"           
[34] "X20.04.BayleyIIILanguageComposite valid_y"           
[35] "X20.04.BayleyIIILanguageComposite valid_percent"     
[36] "X20.04.BayleyIIIMotorComposite corr"                 
[37] "X20.04.BayleyIIIMotorComposite r2"                   
[38] "X20.04.BayleyIIIMotorComposite pvalue"               
[39] "X20.04.BayleyIIIMotorComposite valid"                
[40] "X20.04.BayleyIIIMotorComposite valid_x"              
[41] "X20.04.BayleyIIIMotorComposite valid_y"              
[42] "X20.04.BayleyIIIMotorComposite valid_percent"

In [13]:
.GlobalEnv$EXCLUDE_COLUMNS <- c(y_columns,
    "X01.01.birthNumber", # 1: 525 2: 7
    "X01.01.coreTempBelow32p5COver2H", # 0: 532 (fit criteria),
    "X01.07.pre_CoolbyIceGelPack", # 0: 250, 1: 2

    "X20.09.blindness", # part of the outcome definition
    "X20.09.multipleImpairment", # part of the outcome defition
    "X20.09.moderateSevereCerebralPalsy", # part of the outcome defition
    "X20.09.hearingImpairedWithAid", # part of the outcome defition
    "X20.09.cerebralPalsyMerge", # part of the outcome defition
    "X20.09.afterDischargeSeizure", # part of the outcome defition
    "X20.09.grossMotorFunctionLevelSeverity", # part of the outcome defition

    "X04.01.deathAge_day", # related t the outcome definition

    "X20.10.normalPrimaryOutcome", # part of the outcome defition
    "X20.10.moderateSevereDisabilitySurvivor", # part of the outcome defition
    "X20.10.disabilityLevelSurvivor", # part of the outcome defition
    "X20.10.disabilityLevel", # part of the outcome defition
    "X20.10.outcomeGroup" # part of the outcome defition
)

.GlobalEnv$INCLUDE_COLUMN_PREFIX <- c(
    "01-02",
    "01-03",
    "01-04",
    "01-05",
    "01-05_1",
    "01-05_2",
    "01-06",
    "01-06_1",
    "01-07",
    "01-08",
    "01-09",
    "01-10",
    "01-12",
    "01-12_1",
    "03-01_s",
    "03-02",
    "03-04",
    "03-04_1",
    "03-05",
    "03-05_s",
    "03-05_s1",
    "04-01",
    "04-01_1",
    "04-02",
    "04-03",
    "04-04",
    "04-05",
    "04-06",
    "04-07",
    "04-08",
    "04-09",
    "04-10",
    "04-11",
    "04-12",
    "04-12_1",
    "04-13",
    "04-14",
    "04-15"
)

In [14]:
valid_column <- function(df, column) {
    if (column %in% .GlobalEnv$EXCLUDE_COLUMNS) {
        return(FALSE)
    }

    column_tuple <- COMBINEHarmonizer::flatten_column_tuple(column)

    column_prefix <- column_tuple[[1]]
    var_name <- column_tuple[[2]]

    if (!column_prefix %in% .GlobalEnv$INCLUDE_COLUMN_PREFIX) {
        return(FALSE)
    }

    if (length(grep(pattern = "Bayley", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "death", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "wdrawSupport", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "limitCare", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "\\.followupCenter$", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    # XXX remove Date
    if (length(grep(pattern = "Date\\d*$", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    # XXX remove Time, but retain spontaneousRespirationTime
    if (length(grep(pattern = "Time\\d*$", x = column, perl = TRUE) != 0) &&
            length(grep(pattern = "spontaneousRespirationTime", x = column, perl = TRUE)) == 0) {
        return(FALSE)
    }

    # with only MRINRNPatternOfInjury in MRI.
    if (column_prefix == "03-05") {
        if (var_name != "MRINRNPatternOfInjury") {
            return(FALSE)
        }
    }

    if (class(df[[column]]) != "numeric") {
        message(c("[INFO] invalid dtype: column: ", column, " type: ", class(df[[column]])))
        return(FALSE)
    }

    sum_is_na_column <- sum(!is.na(df[column]))
    if (sum_is_na_column < 20) {
        message(c("[INFO] too few valid samples: column: ", column, " count: ", sum_is_na_column))
        return(FALSE)
    }

    the_list <- df[[column]]
    length_unique <- length(unique(the_list[!is.na(the_list)]))
    if (length_unique <= 1) {
        print(c("[INFO] constant column: ", column))
        return(FALSE)
    }

    return(TRUE)
}

In [15]:
columns <- names(df_numeric_valid)
x_columns <- columns[sapply(columns, \(x) valid_column(df_numeric_valid, x))]
length(x_columns)

[INFO] invalid dtype: column: X01.02.siteID type: character



[INFO] invalid dtype: column: X01.02.screenComment type: character



[1] "[INFO] constant column: "              
[2] "X01.02.coreTempLess32p5CGreaterEq2Hr_e"
[1] "[INFO] constant column: "            
[2] "X01.02.coreTempLess33p5CGreater1Hr_e"
[1] "[INFO] constant column: "           "X01.02.coreTempLess34CGreater1Hr_e"
[1] "[INFO] constant column: "               
[2] "X01.02.first6HrCoolByClinicalProtocol_e"
[1] "[INFO] constant column: "        "X01.02.chromosomalAbnormality_e"
[1] "[INFO] constant column: "        "X01.02.majorCongenitalAnomaly_e"
[1] "[INFO] constant column: "        "X01.02.birthWeightLessEq1800g_e"
[1] "[INFO] constant column: "       "X01.02.infantUnlikelySurvive_e"
[1] "[INFO] constant column: "                                           
[2] "X01.02.first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e"
[1] "[INFO] constant column: "                 
[2] "X01.02.postnatalAgeLess6HrOrGreater24Hr_e"
[1] "[INFO] constant column: "          "X01.02.enrolledConflictingTrial_e"
[1] "[INFO] constant column: "       "X01.02.his

[INFO] invalid dtype: column: X01.02.consentStatus type: character



[INFO] invalid dtype: column: X01.02.noConsentReason type: logical



...

[INFO] invalid dtype: column: X04.14.dischargeBirthDefectCode2 type: character



[INFO] invalid dtype: column: X04.14.dischargeBirthDefectCode3 type: logical



[1] "[INFO] constant column: "                     
[2] "X04.15.dischargeHomeTherapyTemperatureBlanket"


[INFO] invalid dtype: column: X04.15.dischargeHomeTherapyOtherText type: character



[1] 341

In [16]:
x_columns

[1] "X01.02.birthNumber"                                         
  [2] "X01.02.first60MinAnyBloodGasPHLessEq7_i"                    
  [3] "X01.02.first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i"
  [4] "X01.02.ageRand_hr"                                          
  [5] "X01.02.treatmentBlanketType"                                
  [6] "X01.02.treatmentAssignmentDuration_hr"                      
  [7] "X01.02.treatmentAssignmentTemperature"                      
  [8] "X01.02.usualCoolingTreatmentGroup"                          
  [9] "X01.02.inOtherTrial"                                        
 [10] "X01.03.motherAge_year"                                      
 [11] "X01.03.motherRace2"                                         
 [12] "X01.03.motherEducation"                                     
 [13] "X01.03.motherEducation2"                                    
 [14] "X01.03.motherInsurance"                                     
 [15] "X01.03.motherInsurancePublic"                               
 [16] "X01.04.gravida"                                             
 [17] "X01.04.parity"                                              
 [18] "X01.04.multipleBirth"                                       
 [19] "X01.04.prenatalCare"                                        
 [20] "X01.04.hypertensionEclampsia"                               
 [21] "X01.04.antepartumHemorrhage"                                
 [22] "X01.04.thyroidMalfunction"                                  
 [23] "X01.04.diabetes"                                            
 [24] "X01.05.ruptureGreater18Hr"                                  
 [25] "X01.05.ruptureBeforeDelivery"                               
 [26] "X01.05.labor"                                               
 [27] "X01.05.fetalDecelerate"                                     
 [28] "X01.05.cordMishap"                                          
 [29] "X01.05.uterineRupture"                                      
 [30] "X01.05.shoulderDystocia"                                    
 [31] "X01.05.placentalProblem"                                    
 [32] "X01.05.maternalHemorrhage"                                  
 [33] "X01.05.maternalTrauma"                                      
 [34] "X01.05.maternalCardioRespiratoryArrest"                     
 [35] "X01.05.maternalSeizure"                                     
 [36] "X01.05.perinatalSentinelEvent"                              
 [37] "X01.05.pyrexiaGreater37p6C"                                 
 [38] "X01.05.chorioamnionitis"                                    
 [39] "X01.05.placentalPathologyPerformed"                         
 [40] "X01.05.histologicChorioamionitis"                           
 [41] "X01.05.laborAntibiotics"                                    
 [42] "X01.05.emergencyCSection"                                   
 [43] "X01.05_1.cordMishap"                                        
 [44] "X01.05_1.uterineRupture"                                    
 [45] "X01.05_1.placentalProblem"                                  
 [46] "X01.05_1.shoulderDystocia"                                  
 [47] "X01.05_1.maternalHemorrhage"                                
 [48] "X01.05_1.maternalTrauma"                                    
 [49] "X01.05_1.maternalCardioRespiratoryArrest"                   
 [50] "X01.05_1.maternalSeizure"                                   
 [51] "X01.05_1.perinatalSentinelEvent"                            
 [52] "X01.05_2.emergencyCSection"                                 
 [53] "X01.06.encephalopathyLevel"                                 
 [54] "X01.06.randomInfantAge"                                     
 [55] "X01.06.birthWeight_g"                                       
 [56] "X01.06.birthLength_cm"                                      
 [57] "X01.06.birthHeadCircumference_cm"                           
 [58] "X01.06.birthGestationalAge_week"                            
 [59] "X01.06.infantSex"                                  

In [17]:
y_columns

[1] "X20.10.moderateSevereDisabilityOrDeath"
[2] "X20.10.disabilityLevelDeath4Category"  
[3] "X20.10_1.disabilityLevelDeath"         
[4] "X20.04.BayleyIIICognitiveComposite"    
[5] "X20.04.BayleyIIILanguageComposite"     
[6] "X20.04.BayleyIIIMotorComposite"

In [18]:
df_corr_all <- COMBINEHarmonizer::corr(df_numeric_valid, x_columns, y_columns)

df_corr_all_pivot <- data.table::dcast(
    data.table::setDT(df_corr_all),
    x ~ y,
    fun = list(mean),
    value.var = "val"
)

r_column <- "X20.04.BayleyIIICognitiveComposite pvalue"
df_corr_all_pivot[["pvalue_lt_threshold"]] <- df_corr_all_pivot[[r_column]] < COMBINEHarmonizer::P_VALUE_THRESHOLD

all_reorder_columns <- c("x", "pvalue_lt_threshold", reordered_columns)

df_corr_all_pivot_reorder <- df_corr_all_pivot[, ..all_reorder_columns] %>%
    dplyr::arrange(desc(pvalue_lt_threshold), desc(`X20.04.BayleyIIICognitiveComposite r2`))

out_filename <- file.path(out_dir, "31-02-corr-all-R.csv")
write.csv(df_corr_all_pivot_reorder, file = out_filename, row.names = FALSE)

corr: (1/341) x_column: X01.02.birthNumber



corr: (2/341) x_column: X01.02.first60MinAnyBloodGasPHLessEq7_i



corr: (3/341) x_column: X01.02.first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i



corr: (4/341) x_column: X01.02.ageRand_hr



corr: (5/341) x_column: X01.02.treatmentBlanketType



...

corr: (337/341) x_column: X04.15.dischargeHomeTherapyOxygen



corr: (338/341) x_column: X04.15.dischargeHomeTherapyGavageTubeFeed



corr: (339/341) x_column: X04.15.dischargeHomeTherapyGastrostomyTubeFeed



corr: (340/341) x_column: X04.15.dischargeHomeTherapyAnticonvulsantMedication



corr: (341/341) x_column: X04.15.dischargeHomeTherapyOther

